#  Import Statements

In [8]:
import pandas as pd

# Loading Data

In [9]:
df = pd.read_csv('data/data.csv')
cpi = pd.read_csv('data/cpi.csv')
df.head()

,FCID,BookingID,BCreatedDateAEST,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,PriceIncGSTRaw,VYMM,VMakeModel,VMake,VYear,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass
0,1,463259,17/06/2021,708763,Capped,Capped Price - 30K,0,0,180.00,2019 TOYOTA COROLLA,TOYOTA COROLLA,TOYOTA,2019,17885,4102,QLD,Brisbane,1
1,2,1360052,11/01/2024,2122072,Capped,Capped Price - 50K,0,0,315.90,2021 MAZDA CX-5,MAZDA CX-5,MAZDA,2021,17739,2285,NSW,Lake Macquarie,2
2,1,1058706,19/10/2022,1633633,Repair,Replace Wiper Blades,0,0,120.00,2020 SKODA KAROQ,SKODA KAROQ,SKODA,2020,19138,3192,VIC,Melbourne,1
3,2,1078043,11/11/2022,1664447,Logbook,Logbook - 60K / 48m,0,0,462.10,2018 MITSUBISHI ASX,MITSUBISHI ASX,MITSUBISHI,2018,17667,2170,NSW,Sydney,1
4,2,1868175,30/07/2024,3101426,Capped,Capped Price - 30K,0,0,359.21,2020 MAZDA 3,MAZDA 3,MAZDA,2020,17410,3041,VIC,Melbourne,1


# Preprocessing
## What needs to be done?
- Price adjusted for inflation
- one hot encoding?
- train, val, test split

In [10]:
# Price adjustment for inflation
df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST']);
df['Quarter'] = df['BCreatedDateAEST'].dt.to_period('Q');
cpi['Quarter'] = pd.to_datetime(cpi['Quarter']).dt.to_period('Q');
base_cpi = cpi['CPI'].iloc[-1]

/tmp/ipykernel_34044/1965790900.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST']);
/tmp/ipykernel_34044/1965790900.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cpi['Quarter'] = pd.to_datetime(cpi['Quarter']).dt.to_period('Q');


In [11]:

merged_df = df.merge(cpi, on='Quarter');
merged_df['adjusted_price'] = round(merged_df['PriceIncGSTRaw'] * (base_cpi / merged_df['CPI']), 2)


In [12]:
merged_df.head()

,FCID,BookingID,BCreatedDateAEST,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,PriceIncGSTRaw,VYMM,...,VMake,VYear,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,Quarter,CPI,adjusted_price
0,1,463259,2021-06-17,708763,Capped,Capped Price - 30K,0,0,180.00,2019 TOYOTA COROLLA,...,TOYOTA,2019,17885,4102,QLD,Brisbane,1,2021Q2,118.8,211.21
1,2,1360052,2024-01-11,2122072,Capped,Capped Price - 50K,0,0,315.90,2021 MAZDA CX-5,...,MAZDA,2021,17739,2285,NSW,Lake Macquarie,2,2024Q1,137.4,320.50
2,1,1058706,2022-10-19,1633633,Repair,Replace Wiper Blades,0,0,120.00,2020 SKODA KAROQ,...,SKODA,2020,19138,3192,VIC,Melbourne,1,2022Q4,130.8,127.89
3,2,1078043,2022-11-11,1664447,Logbook,Logbook - 60K / 48m,0,0,462.10,2018 MITSUBISHI ASX,...,MITSUBISHI,2018,17667,2170,NSW,Sydney,1,2022Q4,130.8,492.48
4,2,1868175,2024-07-30,3101426,Capped,Capped Price - 30K,0,0,359.21,2020 MAZDA 3,...,MAZDA,2020,17410,3041,VIC,Melbourne,1,2024Q3,139.1,359.98


## Encoding Categoricals